In [1]:
import sys, os
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import sqlalchemy as sql
import sqlite3 as sqlite3

def header(msg):
    print('-'*50 + "\n" + msg +"\n")

filename='GSL_Ops.xlsx'   
conn = sqlite3.connect('customer.db')
cur=conn.cursor()


In [2]:
#header('What’s the revenue of the 90th percentile rank customer_id from CustomerRevenue, ordered by revenue (where the highest-revenue customer_id is in the 99th percentile)?')
sql_problem_1 = ("SELECT '90th Percentile' as Percentile_Rank, min(revenue) as revenue "
                 "FROM (Select CR.customer_id,printf('%.3f',CR.revenue) as revenue, PERCENT_RANK() Over(Order By CR.revenue asc) as rank "
                 "From CustomerRevenue as CR "
                 "Order by CR.revenue asc) "
                 "WHERE rank >=.9 "
                 "Group by 1;").format(1, 2)

print(sql_problem_1)
pd.read_sql_query(sql_problem_1,conn)




SELECT '90th Percentile' as Percentile_Rank, min(revenue) as revenue FROM (Select CR.customer_id,printf('%.3f',CR.revenue) as revenue, PERCENT_RANK() Over(Order By CR.revenue asc) as rank From CustomerRevenue as CR Order by CR.revenue asc) WHERE rank >=.9 Group by 1;


,Percentile_Rank,revenue
0,90th Percentile,6426.570


In [8]:
#header('What is the revenue by sales team region, along with revenue from accounts that have no sales team (note: Some customer IDs are assigned to a sales team in CustomerSalesTeam. Use this relationship to find your answer)?')
sql_problem_2 = ("Select coalesce(CST.region,'No Sales Team') as Sales_Team_region, sum(printf('%.3f',CR.revenue)) as revenue "
                 "From CustomerRevenue as CR "
                 "Left Join CustomerSalesTeam CST on CR.customer_id = CST.customer_id "
                 "Group by 1;").format(1, 2)

print(sql_problem_2)
pd.read_sql_query(sql_problem_2,conn)




Select coalesce(CST.region,'No Sales Team') as Sales_Team_region, sum(printf('%.3f',CR.revenue)) as revenue From CustomerRevenue as CR Left Join CustomerSalesTeam CST on CR.customer_id = CST.customer_id Group by 1;


,Sales_Team_region,revenue
0,APAC,46220.662
1,Americas,68450.476
2,EMEA,89071.016
3,No Sales Team,1976839.176


In [3]:
header('What is the revenue per agency (note: Some customer IDs have an agency assigned, in addition to their CustomerSalesTeam. A customer ID has an agency sales team if it both (1) appears in CustomerAgency and (2) the customer ID’s country in CustomerAgency matches the customer ID’s country in CustomerSalesTeam)?')
sql_problem_3 = ("Select CA.agency_name , sum(printf('%.3f',CR.revenue)) as revenue "
                     "FROM CustomerAgency CA "
                     "INNER JOIN CustomerSalesTeam CST on CA.customer_id = CST.customer_id and CA.country=CST.country "
                "LEFT JOIN CustomerRevenue CR on CA.customer_id = CR.customer_id "
                     "GROUP BY 1;").format(1,2)
print(sql_problem_3)
pd.read_sql_query(sql_problem_3,conn)


--------------------------------------------------
What is the revenue per agency (note: Some customer IDs have an agency assigned, in addition to their CustomerSalesTeam. A customer ID has an agency sales team if it both (1) appears in CustomerAgency and (2) the customer ID’s country in CustomerAgency matches the customer ID’s country in CustomerSalesTeam)?

Select CA.agency_name , sum(printf('%.3f',CR.revenue)) as revenue FROM CustomerAgency CA INNER JOIN CustomerSalesTeam CST on CA.customer_id = CST.customer_id and CA.country=CST.country LEFT JOIN CustomerRevenue CR on CA.customer_id = CR.customer_id GROUP BY 1;


,agency_name,revenue
0,Dentsu,26404.907
1,Independents,29202.143
2,Omnicom,13360.477
3,Publicis,10570.326
4,WPP,22722.817


In [4]:
header("Google is always running experiments to try to optimize our products. The CustomerExperiments table indicates which customer IDs are in the ALPHA experiment and which are in the BRAVO experiment. (Some customer IDs are enrolled in both.)"
       "\n When comparing total revenue from customer IDs in the ALPHA experiment to total revenue from customer IDs not in the experiment, which cohort has higher revenue?"
       "\n Is revenue from customers enrolled in the ALPHA experiment significantly different than revenue from customers not in the experiment? What about the BRAVO experiment? Explain the test that you used and include any database queries you would have used to generate the inputs required for the test.")

--------------------------------------------------
Google is always running experiments to try to optimize our products. The CustomerExperiments table indicates which customer IDs are in the ALPHA experiment and which are in the BRAVO experiment. (Some customer IDs are enrolled in both.)
 When comparing total revenue from customer IDs in the ALPHA experiment to total revenue from customer IDs not in the experiment, which cohort has higher revenue?
 Is revenue from customers enrolled in the ALPHA experiment significantly different than revenue from customers not in the experiment? What about the BRAVO experiment? Explain the test that you used and include any database queries you would have used to generate the inputs required for the test.



In [13]:
print("Cohort with with higher")
sql_problem_4 = ("Select CASE WHEN CE.experiment_id = 'ALPHA' then 'ALPHA' else 'NOT_ALPHA' end as experiment_id , sum(printf('%.3f',CR.revenue)) as revenue "
                     "FROM CustomerRevenue CR "
                     "LEFT JOIN CustomerExperiment CE on CR.customer_id = CE.customer_id "
                     "GROUP BY 1;").format(1, 2)
print(sql_problem_4)
pd.read_sql_query(sql_problem_4,conn)

Cohort with with higher
Select CASE WHEN CE.experiment_id = 'ALPHA' then 'ALPHA' else 'NOT_ALPHA' end as experiment_id , sum(printf('%.3f',CR.revenue)) as revenue FROM CustomerRevenue CR LEFT JOIN CustomerExperiment CE on CR.customer_id = CE.customer_id GROUP BY 1;


,experiment_id,revenue
0,ALPHA,1191797.061
1,NOT_ALPHA,1627826.360


In [23]:
print("Breakdown of metrics between ALPHA and Bravo")
sql_problem_4 = ("SELECT QUARTILE, "
                     "MIN(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as quartile_customer_rev_ALPHA, "
                     "MIN(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as quartile_customer_rev_BRAVO, "
                     "MIN(CASE WHEN experiment_id is null then revenue else null end) as quartile_customer_rev_NONE, "                 
                     "AVG(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as average_customer_rev_ALPHA, "
                     "AVG(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as average_customer_rev_BRAVO, "
                     "AVG(CASE WHEN experiment_id is null then revenue else null end) as average_customer_rev_NONE, "                 
                     "SUM(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as total_rev_ALPHA, "
                     "SUM(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as total_rev_BRAVO, "
                     "SUM(CASE WHEN experiment_id is null then revenue else null end) as total_rev_NONE, "                 
                     "COUNT(CASE WHEN experiment_id = 'ALPHA' then customer_id else null end) as participants_ALPHA, "
                     "COUNT(CASE WHEN experiment_id = 'BRAVO' then customer_id else null end) as participants_BRAVO, "
                     "COUNT(CASE WHEN experiment_id is null then customer_id else null end) as participants_NONE "                 
                     "FROM(SELECT experiment_id, customer_id, revenue, rev_rank, "
                      "CASE WHEN rev_rank >=0 AND rev_rank<.25 then '1st Quartile'"
                           "WHEN rev_rank >=.25 AND rev_rank<.5 then '2nd Quartile' "
                           "WHEN rev_rank >=.5 AND rev_rank<.75 then '3rd Quartile' "
                           "WHEN rev_rank >=.75 AND rev_rank<=1 then '4th Quartile' "
                           "END as Quartile "
                          "FROM(SELECT experiment_id, customer_id, revenue, percent_rank() Over(partition by experiment_id ORDER BY revenue asc) as rev_rank "
                            "FROM(Select CE.experiment_id , coalesce(CE.customer_id, CR.customer_id) as customer_id, "
                                 "sum(printf('%.3f',CR.revenue)) as revenue "
                             "FROM CustomerRevenue CR "
                             "LEFT JOIN CustomerExperiment CE on CE.customer_id = CR.customer_id "
                             "GROUP BY 1,2) "
                         "GROUP BY 1,2,3) "                 
                     "GROUP BY 1,2,3,4,5 ORDER BY 1,4,5,2) "
                 "Group by 1;").format(1, 2)
print(sql_problem_4)
pd.read_sql_query(sql_problem_4,conn)

Breakdown of metrics between ALPHA and Bravo
SELECT QUARTILE, MIN(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as quartile_customer_rev_ALPHA, MIN(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as quartile_customer_rev_BRAVO, MIN(CASE WHEN experiment_id is null then revenue else null end) as quartile_customer_rev_NONE, AVG(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as average_customer_rev_ALPHA, AVG(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as average_customer_rev_BRAVO, AVG(CASE WHEN experiment_id is null then revenue else null end) as average_customer_rev_NONE, SUM(CASE WHEN experiment_id = 'ALPHA' then revenue else null end) as total_rev_ALPHA, SUM(CASE WHEN experiment_id = 'BRAVO' then revenue else null end) as total_rev_BRAVO, SUM(CASE WHEN experiment_id is null then revenue else null end) as total_rev_NONE, COUNT(CASE WHEN experiment_id = 'ALPHA' then customer_id else null end) as participants_ALPHA, COUNT(CAS

,Quartile,quartile_customer_rev_ALPHA,quartile_customer_rev_BRAVO,quartile_customer_rev_NONE,average_customer_rev_ALPHA,average_customer_rev_BRAVO,average_customer_rev_NONE,total_rev_ALPHA,total_rev_BRAVO,total_rev_NONE,participants_ALPHA,participants_BRAVO,participants_NONE
0,1st Quartile,100.358,3394.886,103.776,201.973092,3884.283959,166.832135,26458.475,190329.914,16015.885,131,49,96
1,2nd Quartile,355.050,4426.047,250.632,641.990308,5034.161347,388.294177,83458.740,246673.906,37276.241,130,49,96
2,3rd Quartile,1038.897,5699.767,567.263,2030.832200,6381.875878,1007.461760,264008.186,312711.918,96716.329,130,49,96
3,4th Quartile,3450.168,7023.366,1478.211,6243.295115,8410.145400,3204.113510,817871.660,420507.270,307594.897,131,50,96


In [21]:
print("Breakdown of metrics between ALPHA and Bravo")
sql_problem_4 = ("Select coalesce(CE.customer_id, CR.customer_id) as customer_id, CE.experiment_id ,  "
                             "sum(printf('%.3f',CR.revenue)) as revenue "
                             "FROM CustomerRevenue CR "
                             "LEFT JOIN CustomerExperiment CE on CE.customer_id = CR.customer_id "
                             "GROUP BY 1,2").format(1, 2)
print(sql_problem_4)
pd.read_sql_query(sql_problem_4,conn)

Breakdown of metrics between ALPHA and Bravo
Select coalesce(CE.customer_id, CR.customer_id) as customer_id, CE.experiment_id ,  sum(printf('%.3f',CR.revenue)) as revenue FROM CustomerRevenue CR LEFT JOIN CustomerExperiment CE on CE.customer_id = CR.customer_id GROUP BY 1,2


,customer_id,experiment_id,revenue
0,104,None,142.839
1,327,None,417.889
2,430,ALPHA,1290.285
3,538,None,267.070
4,585,ALPHA,349.064
5,775,ALPHA,501.019
6,823,ALPHA,108.366
7,961,None,256.016
8,997,ALPHA,253.604
9,1102,ALPHA,764.636
